<a href="https://colab.research.google.com/github/Amanj5486/Finance_ML/blob/main/Tesla_Tf_basedon_last6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import math
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import confusion_matrix, accuracy_score


In [40]:
df=pd.read_csv('TSLA.csv',index_col='Date',parse_dates=True)

In [41]:
data=df.filter(['Adj Close'])
dataset=data.values
training_data_len=math.ceil(len(dataset)*0.8)
dataset

array([[  5.72    ],
       [  5.464   ],
       [  5.3     ],
       ...,
       [598.780029],
       [610.119995],
       [609.890015]])

In [42]:
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)
scaled_data

array([[0.0015136 ],
       [0.00122226],
       [0.00103562],
       ...,
       [0.67644248],
       [0.68934787],
       [0.68908615]])

In [43]:
train_data=scaled_data[0:training_data_len,:]
x_train=[]
y_train=[]
for i in range(60,len(train_data)):
  x_train.append(train_data[i-60:i,0])
  y_train.append(train_data[i,0])

In [44]:
x_train,y_train=np.array(x_train),np.array(y_train)
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
x_train.shape

(1953, 60, 1)

In [45]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(LSTM(50,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [46]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [47]:
model.fit(x_train,y_train,batch_size=1,epochs=1)

1953/1953 [==============================] - 52s 25ms/step - loss: 6.8016e-05


In [48]:
test_data=scaled_data[training_data_len-60:,:]
x_test=[]
y_test=dataset[training_data_len:,:]
for i in range(60,len(test_data)):
  x_test.append(test_data[i-60:i,0])

In [49]:
x_test=np.array(x_test)

In [50]:
x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

In [51]:
predictions=model.predict(x_test)
predictions=scaler.inverse_transform(predictions)


In [52]:
rmse=np.sqrt(np.mean(predictions-y_test)**2)
rmse

85.59370204261928

In [53]:
valid=data[training_data_len:]
valid['Predictions']=predictions

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [54]:
valid

,Adj Close,Predictions
Date,,
2019-06-14,42.984001,42.044746
2019-06-17,45.006001,42.300823
2019-06-18,44.948002,42.721329
2019-06-19,45.285999,43.143749
2019-06-20,43.924000,43.539425
...,...,...
2021-06-07,605.130005,431.073730
2021-06-08,603.590027,429.393188
2021-06-09,598.780029,428.654785


In [55]:
r2=sklearn.metrics.r2_score(y_test, predictions)
r2

0.7729893525779692